In [173]:
from DataStructure import *
from OrderType import *
import numpy as np

exchange = Exchange()
exchange.addAsset('TEST')

def generate_valid_price(base_mean, base_std, islong, cluster_std=3):
    while True:  
        if np.random.rand() < 0.8:  
            price = np.random.normal(loc=base_mean, scale=base_std)
        else:  
            if islong:
                cluster_center = base_mean + np.random.choice(range(-140, 0, 10)) 
                price = np.random.normal(loc=cluster_center, scale=cluster_std)
            else:
                cluster_center = base_mean + np.random.choice(range(0, 140, 10)) 
                price = np.random.normal(loc=cluster_center, scale=cluster_std)

        if price >= 1: 
            return price

for i in range(100000):
    if np.random.choice((0, 1)) == 0: 
        price = generate_valid_price(base_mean=160, base_std=30, islong=False)
        exchange.ex_trade(LimitShort(np.random.randint(10, 40), 'TEST', np.round(price, 0)))
    else:  
        price = generate_valid_price(base_mean=140, base_std=30, islong=True)
        exchange.ex_trade(LimitLong(np.random.randint(10, 40), 'TEST', np.round(price, 0)))


In [174]:
orderbook = exchange.getOrderBook('TEST')

In [175]:
long_erg = {}
short_erg = {}

def scrapePriceNodes(result: dict, node: PriceNode):
    if node == None:
        return
    result[node.price] = node.orderlist.getOrderVolume()

    if node.left != None:
        scrapePriceNodes(result, node.left)
    if node.right != None:
        scrapePriceNodes(result, node.right)

    return

In [176]:
scrapePriceNodes(long_erg, node=orderbook.long_limit.root)
scrapePriceNodes(short_erg, node=orderbook.short_limit.root)

In [177]:
matching_dict = {}
limit_dict = {}


all_keys = set(short_erg.keys()).union(long_erg.keys())

for key in all_keys:
    value1 = short_erg.get(key, 0) 
    value2 = long_erg.get(key, 0) 
    matching_dict[key] = min(value1, value2)  
    limit_dict[key] = abs(value1-value2)

max_price = max(matching_dict, key=matching_dict.get)  
max_value = matching_dict[max_price]   

In [182]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Bar(
            x=list(matching_dict.keys()),  
            y=list(matching_dict.values()), 
            text=list(matching_dict.values()),
            textposition='auto',  
            marker_color='green',
            name='Matched Volume'
        ),
        go.Bar(
            x=list(limit_dict.keys()),  
            y=list(limit_dict.values()), 
            text=list(limit_dict.values()),
            textposition='auto',  
            marker_color='black',
            name='Outstanding Volume'
        )
    ]
)

fig.add_shape(
    type='line',
    xref='x',  
    yref='y', 
    x0=max_price,
    x1=max_price,
    y0=0,
    y1=max_value,
    line=dict(color='red', width=2, dash='dash')
)

fig.update_layout(
    title="Volume/Matched Volume per Limit Price",
    xaxis_title="Limit Price",
    yaxis_title="Volume/Matched Volume",
    template="plotly_white"
)

fig.show()